<a href="https://colab.research.google.com/github/omnimanwani/DesignProject/blob/main/Inetent%2Bentity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jsonlines

In [ ]:
!pip install jsonlines
!pip install sentence-transformers
!pip install tmdbv3api
!pip install spacy
!python -m spacy download en_core_web_lg

from dataclasses import dataclass
from typing import List, Dict
import jsonlines
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
from tmdbv3api import TMDb
from tmdbv3api import Movie
import spacy
import pandas as pd
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install transformers torch spacy sentence-transformers tmdbv3api jsonlines pandas
!python -m spacy download en_core_web_lg

from dataclasses import dataclass
from typing import List, Dict
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from tmdbv3api import TMDb, Movie, Genre
import jsonlines
import pandas as pd
import json
from SPARQLWrapper import SPARQLWrapper, JSON

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 931.2 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import requests

In [ ]:
from tmdbv3api import Discover, Genre

In [ ]:
from typing import Dict, List, Optional
from SPARQLWrapper import SPARQLWrapper, JSON
from dataclasses import dataclass
import time
from urllib.parse import quote

In [ ]:
class DBpediaQuerier:
    def __init__(self):
        self.sparql = SPARQLWrapper("http://dbpedia.org/sparql")

    def query_person_role(self, person_name: str) -> Dict[str, str]:
        """Query DBpedia to determine if a person is a director, writer, or actor"""
        query = """
        SELECT DISTINCT ?role WHERE {
          ?person rdfs:label ?name .
          FILTER(CONTAINS(LCASE(?name), LCASE("%s")))
          {
            ?movie dbo:director ?person .
            BIND("director" AS ?role)
          } UNION {
            ?movie dbo:writer ?person .
            BIND("writer" AS ?role)
          } UNION {
            ?movie dbo:starring ?person .
            BIND("actor" AS ?role)
          }
        }
        """ % person_name

        self.sparql.setQuery(query)
        self.sparql.setReturnFormat(JSON)

        try:
            results = self.sparql.query().convert()
            roles = [result['role']['value'] for result in results['results']['bindings']]
            return {
                'name': person_name,
                'roles': roles
            }
        except Exception as e:
            print(f"DBpedia query error for {person_name}: {e}")
            return {'name': person_name, 'roles': []}

In [ ]:
@dataclass
class EntityInfo:
    director: List[str] = None
    writer: List[str] = None
    cast: List[str] = None
    genre: List[str] = None
    year: List[str] = None
    movie_name: List[str] = None

    def __post_init__(self):
        if self.director is None:
            self.director = []
        if self.writer is None:
            self.writer = []
        if self.genre is None:
            self.genre = []
        if self.cast is None:
            self.cast = []
        if self.year is None:
            self.year = []
        if self.movie_name is None:
            self.movie_name = []

@dataclass
class MessageAnalysisResult:
    message_id: int
    conversation_id: str
    text: str
    time_offset: int
    intent: str
    confidence: float
    entities: EntityInfo
    referenced_movies: List[str]

class FewShotMovieDialogAnalyzer:
    def __init__(self, model_name='sentence-transformers/all-MiniLM-L6-v2', tmdb_api_key='eb1dc87b7306476a4200d6c4e88d75d4'):
        self._setup_model(model_name)
        self._initialize_examples()

        self.tmdb_enabled = False
        self.genre_names = {}
        if tmdb_api_key:
            try:
                self.tmdb = TMDb()
                self.tmdb.api_key = tmdb_api_key
                self.movie_db = Movie()
                self.genre = Genre()
                self.tmdb_enabled = True

                self.genre_list = self.genre.movie_list()
                self.genre_names = {genre['name'].lower(): genre['id']
                                  for genre in self.genre_list['genres']}
            except Exception as e:
                print(f"Warning: Failed to initialize TMDB: {e}")

        self.nlp = spacy.load("en_core_web_lg")

        self.dbpedia_querier = DBpediaQuerier()

        self.entity_mappings = {
            "PERSON": ["director", "writer", "cast"],
            "DATE": "year",
            "WORK_OF_ART": "movie_name",
            "NORP": "genre",
            "GPE": "genre"
        }

        self.entity_examples = {
            'director': ["directed", "director"],
            'writer': ["written by", "screenplay", "writer"],
            'cast': ["starring", "actor", "actress", "lead"],
            'genre': ["science fiction", "action thriller", "comedy", "sci-fi", "romantic"],
            'year': ["(20", "(19"],
            'movie_name': ["The", "Redemption", "Knight", "Fiction"]
        }

    def _get_movie_details_from_tmdb(self, movie_title: str, year: str = None) -> dict:
        if not self.tmdb_enabled:
            return None

        try:
            search_query = movie_title
            if year:
                search_query += f" ({year})"

            search_results = self.movie_db.search(search_query)

            if not search_results:
                return None

            movie = search_results[0]

            details = self.movie_db.details(movie.id)

            credits = self.movie_db.credits(movie.id)

            return {
                'title': details.title,
                'year': details.release_date[:4] if details.release_date else None,
                'directors': [crew['name'] for crew in credits['crew'] if crew['job'] == 'Director'],
                'writers': [crew['name'] for crew in credits['crew'] if crew['job'] in ['Screenplay', 'Writer']],
                'cast': [cast['name'] for cast in credits['cast'][:5]],  # Top 5 cast members
                'genres': [genre.name for genre in details.genres]
            }
        except Exception as e:
            print(f"Error fetching TMDB data: {e}")
            return None

    def extract_entities(self, text: str, movie_mentions: Dict = None) -> EntityInfo:
          extracted_entities = EntityInfo()
          doc = self.nlp(text)

          person_after_role = None
          previous_token = None

          for token in doc:
              if token.text.lower() in ["directed", "director"]:
                  person_after_role = "director"
              elif token.text.lower() in ["written", "writer", "screenplay"]:
                  person_after_role = "writer"
              elif token.text.lower() in ["starring", "stars", "featuring"]:
                  person_after_role = "cast"

              elif person_after_role and token.pos_ == "PROPN":
                  if previous_token and previous_token.pos_ == "PROPN":
                      person_name = f"{previous_token.text} {token.text}"
                  else:
                      person_name = token.text

                  if person_after_role == "director" and not extracted_entities.director:
                      extracted_entities.director = [person_name]
                  elif person_after_role == "writer" and not extracted_entities.writer:
                      extracted_entities.writer = [person_name]
                  elif person_after_role == "cast":
                      extracted_entities.cast.append(person_name)

                  person_after_role = None

              previous_token = token

          for ent in doc.ents:
              mapped_fields = self.entity_mappings.get(ent.label_)

              if isinstance(mapped_fields, list):
                  for field_name in mapped_fields:
                      if field_name == "cast":
                          extracted_entities.cast.append(ent.text)
                      elif field_name == "director" and not extracted_entities.director:
                          extracted_entities.director = [ent.text]
                      elif field_name == "writer" and not extracted_entities.writer:
                          extracted_entities.writer = [ent.text]
              elif mapped_fields:
                  if mapped_fields == "year":
                      extracted_entities.year.append(ent.text)
                  elif mapped_fields == "movie_name":
                      extracted_entities.movie_name.append(ent.text)
                  elif mapped_fields == "genre":
                      extracted_entities.genre = [ent.text]

          if movie_mentions:
              for movie_id, movie_info in movie_mentions.items():
                  if f"@{movie_id}" in text:
                      movie_parts = movie_info.split(" (")
                      if len(movie_parts) == 2:
                          title = movie_parts[0].strip()
                          year = movie_parts[1].replace(")", "").strip()

                          if title and title not in extracted_entities.movie_name:
                              extracted_entities.movie_name.append(title)
                          if year and year not in extracted_entities.year:
                              extracted_entities.year.append(year)

          for name in extracted_entities.director or []:
              person_info = self.dbpedia_querier.query_person_role(name)
              if 'director' not in person_info['roles']:
                  extracted_entities.director = None

          for name in extracted_entities.writer or []:
              person_info = self.dbpedia_querier.query_person_role(name)
              if 'writer' not in person_info['roles']:
                  extracted_entities.writer = None

          for name in extracted_entities.cast[:]:
              person_info = self.dbpedia_querier.query_person_role(name)
              if 'actor' not in person_info['roles']:
                  extracted_entities.cast.remove(name)

          if not extracted_entities.genre and movie_mentions and self.tmdb_enabled:
              for movie_id, movie_info in movie_mentions.items():
                  if f"@{movie_id}" in text:
                      movie_parts = movie_info.split(" (")
                      if len(movie_parts) >= 1:
                          title = movie_parts[0].strip()
                          try:
                              search = self.movie_db.search(title)
                              if search:
                                  movie_details = self.movie_db.details(search[0].id)
                                  genres = [genre['name'] for genre in movie_details.genres]
                                  extracted_entities.genre = genres
                          except Exception as e:
                              print(f"TMDB genre lookup error for {title}: {e}")

          if not extracted_entities.genre and self.tmdb_enabled:
              doc_lower = text.lower()
              found_genres = []
              for genre_name in self.genre_names:
                  if genre_name in doc_lower:
                      found_genres.append(genre_name.title())
              if found_genres:
                  extracted_entities.genre = found_genres

          extracted_entities.movie_name = list(dict.fromkeys(extracted_entities.movie_name))
          extracted_entities.year = list(dict.fromkeys(extracted_entities.year))
          extracted_entities.cast = list(dict.fromkeys(extracted_entities.cast))

          return extracted_entities


    def _setup_model(self, model_name: str) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        if torch.cuda.is_available():
            self.model = self.model.to('cuda')

    def _initialize_examples(self) -> None:
        """Initialize few-shot examples for intent and entity recognition."""
        self.intent_examples = {
            'chit-chat': [
                "Hey, how's it going?",
                "I love watching movies on weekends!",
                "Movies are my favorite way to relax",
                "I had such a great time watching movies yesterday",
                "What kind of movies do you usually watch?",
                "I've been really into movies lately"
            ],
            'query': [
                "Who directed Inception?",
                "What's the plot of The Matrix about?",
                "When was this movie released?",
                "Is this movie available on Netflix?",
                "How long is the movie?",
                "What's the rating for this film?"
            ],
            'recommend': [
                "Can you suggest a good thriller movie?",
                "I'm looking for something similar to Interstellar",
                "What should I watch if I liked The Dark Knight?",
                "Please recommend some good sci-fi movies",
                "I need a movie suggestion for tonight",
                "What are some good movies like this one?"
            ]
        }

    def get_embeddings(self, texts: List[str]) -> np.ndarray:
        inputs = self.tokenizer(texts, padding=True, truncation=True,
                              max_length=512, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = {k: v.to('cuda') for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model(**inputs)

        embeddings = torch.mean(outputs.last_hidden_state, dim=1)
        return embeddings.cpu().numpy()

    def classify_intent(self, text: str) -> Dict[str, float]:
        text_embedding = self.get_embeddings([text])

        best_score = -1
        predicted_intent = None

        for intent, examples in self.intent_examples.items():
            example_embeddings = self.get_embeddings(examples)
            similarities = cosine_similarity(text_embedding, example_embeddings)
            avg_similarity = np.mean(similarities)

            if avg_similarity > best_score:
                best_score = avg_similarity
                predicted_intent = intent

        return {
            "intent": predicted_intent,
            "confidence": float(best_score)
        }

    def _process_dialogue(self, dialogue: Dict) -> List[MessageAnalysisResult]:
        results = []
        movie_mentions = dialogue.get('movieMentions', {})
        initiator_id = dialogue.get('initiatorWorkerId')

        for message in dialogue.get('messages', []):
            if message.get('senderWorkerId') == initiator_id:
                processed_text = self._replace_movie_tokens(message.get('text', ''), movie_mentions)

                if len(processed_text.strip()) < 2:
                    continue

                referenced_movies = self._extract_referenced_movies(
                    message.get('text', ''),
                    movie_mentions
                )

                result = self._analyze_message(
                    message_id=message.get('messageId', 0),
                    conversation_id=dialogue.get('conversationId', ''),
                    text=processed_text,
                    original_text=message.get('text', ''),
                    time_offset=message.get('timeOffset', 0),
                    referenced_movies=referenced_movies,
                    movie_mentions=movie_mentions
                )

                results.append(result)

        return results

    def _replace_movie_tokens(self, text: str, movie_mentions: Dict[str, str]) -> str:
        processed_text = text
        for movie_id, movie_name in movie_mentions.items():
            processed_text = processed_text.replace(f"@{movie_id}", movie_name)
        return processed_text

    def _extract_referenced_movies(self, text: str, movie_mentions: Dict[str, str]) -> List[str]:
        referenced_movies = []
        for movie_id, movie_name in movie_mentions.items():
            if f"@{movie_id}" in text:
                referenced_movies.append(movie_name)
        return referenced_movies

    def _analyze_message(self, message_id: int, conversation_id: str, text: str,
                        original_text: str, time_offset: int,
                        referenced_movies: List[str], movie_mentions: Dict) -> MessageAnalysisResult:
        intent_results = self.classify_intent(text)
        entities = self.extract_entities(original_text, movie_mentions)

        return MessageAnalysisResult(
            message_id=message_id,
            conversation_id=conversation_id,
            text=original_text,
            time_offset=time_offset,
            intent=intent_results['intent'],
            confidence=intent_results['confidence'],
            entities=entities,
            referenced_movies=referenced_movies
        )

    def process_jsonl_file(self, input_file: str, batch_size: int = 4) -> List[MessageAnalysisResult]:
        all_results = []

        with jsonlines.open(input_file) as reader:
            dialogues = list(reader)

        for start in range(0, len(dialogues), batch_size):
            batch = dialogues[start:start + batch_size]
            for dialogue in batch:
                dialogue_results = self._process_dialogue(dialogue)
                all_results.extend(dialogue_results)

        return all_results

    def export_results_to_csv(self, results: List[MessageAnalysisResult], output_file: str) -> None:
        data = [{
            "message_id": result.message_id,
            "conversation_id": result.conversation_id,
            "text": result.text,
            "time_offset": result.time_offset,
            "intent": result.intent,
            "intent_confidence": result.confidence,
            "director": result.entities.director,
            "writer": result.entities.writer,
            "cast": ','.join(result.entities.cast) if result.entities.cast else None,
            "genre": result.entities.genre,
            "year": ','.join(result.entities.year) if result.entities.year else None,
            "movie_name": ','.join(result.entities.movie_name) if result.entities.movie_name else None,
            "referenced_movies": json.dumps(result.referenced_movies),
        } for result in results]

        df = pd.DataFrame(data)
        df.to_csv(output_file, index=False)


def main():
    input_file = 'train.jsonl'
    output_file = 'output_results.csv'

    analyzer = FewShotMovieDialogAnalyzer()

    print(f"Processing the input file: {input_file}")
    results = analyzer.process_jsonl_file(input_file)

    print(f"Exporting results to: {output_file}")
    analyzer.export_results_to_csv(results, output_file)

    print(f"Processing completed. Results saved to {output_file}")

if __name__ == "__main__":
    main()

Processing the input file: train.jsonl
Exporting results to: output_results.csv
Processing completed. Results saved to output_results.csv
